<a href="https://colab.research.google.com/github/OmdenaAI/SudanChapter_AnalyzeHealthcareAccessibility/blob/main/04_Data_analysis/Geographic/Omedena_geographic_segment_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Imoprting libraries<\b>

In [ ]:
import ast
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

<b>Reading CSV<\b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import gdown

file_url = "https://drivergoogle.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR"

output = "geographic_merged.csv"

gdown.download(file_url, output, quiet=False)

In [ ]:

file_path = "/content/geographic_merged.csv"
geo = pd.read_csv(file_path)

<b>EDA<\b>

In [ ]:
geo.head()

In [ ]:
geo.info()

In [ ]:
geo.nunique()

In [ ]:
# Converts each record to python literal from string objects

geo["SENTINEL_2_NDVI_L3"] = geo["SENTINEL_2_NDVI_L3"].apply(lambda x: ast.literal_eval(x))
geo["SENTINEL_2_NDWI_L3"] = geo["SENTINEL_2_NDWI_L3"].apply(lambda x: ast.literal_eval(x))
geo["WEATHER_WEEKLY_L1"] = geo["WEATHER_WEEKLY_L1"].apply(lambda x: ast.literal_eval(x))

In [ ]:
# Flattening function

def flatten_sentinel_data(df, column, region_col, geom_col):
    records = []
    for idx, row in df.iterrows():
        if isinstance(row[column], dict):
            for key, values in row[column].items():
                for item in values:
                    records.append({
                        'date': pd.to_datetime(item['interval_from']),
                        'value': item['ndvi_mean'],
                        'region': row[region_col],
                        'geometry': row[geom_col]
                    })
    return pd.DataFrame(records)

In [ ]:
geo.columns

In [ ]:

# Flattening relevant columns

ndvi_data = flatten_sentinel_data(geo, 'SENTINEL_2_NDVI_L3', 'NAME_L3', 'GEOMETRY_L3')
ndwi_data = flatten_sentinel_data(geo, 'SENTINEL_2_NDWI_L3', 'NAME_L3', 'GEOMETRY_L3')

In [ ]:
ndvi_data.columns

In [ ]:
ndvi_data.head()

In [ ]:
# Concating features
geo_data=pd.concat([geo,ndvi_data ,ndwi_data], axis=1)
geo_data.head()